<a href="https://colab.research.google.com/github/olaakintola/WheelSpace/blob/main/allroutesplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ROUTE MACHINE LEARNING MODEL - picked the alternating first 3 into train, and next 1 into test - smote

# The entire bus route was used in creating the model. The datasets were split into training and test 
# dataset before it was read into colab. The first 3 files were picked for training datasets and the next 
# csv file was used for test dataset. The process was repeated like this until all the files had been
# divided into training and test dataset. SMOTE Sampling technique was used for improving the training datasets.
# Random Forest classifier was used here.

!pip install -q sklearn

In [2]:
%tensorflow_version 2.x

In [3]:
from __future__ import absolute_import, division, print_function,unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
training_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/traindata.csv")
testing_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/testdata.csv")

In [5]:
training_df.columns

Index(['route', 'times', 'days', 'wheeloccupancy', 'departure', 'destination'], dtype='object')

In [6]:
testing_df.columns

Index(['route', 'times', 'days', 'wheeloccupancy', 'departure', 'destination'], dtype='object')

In [7]:
training_df["wheeloccupancy"]

0         0.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
109076    0.0
109077    0.0
109078    0.0
109079    0.0
109080    NaN
Name: wheeloccupancy, Length: 109081, dtype: float64

In [8]:
testing_df["wheeloccupancy"]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
36356    0.0
36357    0.0
36358    0.0
36359    0.0
36360    NaN
Name: wheeloccupancy, Length: 36361, dtype: float64

In [9]:
training_df["wheeloccupancy"] = training_df["wheeloccupancy"].fillna(0).astype(int)

In [10]:
testing_df["wheeloccupancy"] = testing_df["wheeloccupancy"].fillna(0).astype(int)

In [11]:
training_df["wheeloccupancy"]

0         0
1         0
2         0
3         0
4         1
         ..
109076    0
109077    0
109078    0
109079    0
109080    0
Name: wheeloccupancy, Length: 109081, dtype: int64

In [12]:
testing_df["wheeloccupancy"]

0        0
1        0
2        0
3        0
4        0
        ..
36356    0
36357    0
36358    0
36359    0
36360    0
Name: wheeloccupancy, Length: 36361, dtype: int64

In [13]:
training_df.shape

(109081, 6)

In [14]:
testing_df.shape

(36361, 6)

In [15]:
training_df.sample(5)

,route,times,days,wheeloccupancy,departure,destination
56892,33E,07:00,tues,0,NaN,NaN
57429,4,16:00,mon,0,NaN,NaN
72815,11,18:00,wed,0,NaN,NaN
86742,29A,13:00,thurs,0,NaN,NaN
26944,38D,11:00,mon,0,NaN,NaN


In [16]:
testing_df.sample(5)

,route,times,days,wheeloccupancy,departure,destination
15834,44B,13:00,fri,0,NaN,NaN
31160,150,15:00,tues,0,NaN,NaN
34981,66X,08:00,mon,0,NaN,NaN
23718,7A,13:00,tues,0,NaN,NaN
24655,123,14:00,fri,0,NaN,NaN


In [17]:
training_df.fillna('', inplace=True)

In [18]:
testing_df.fillna('', inplace=True)

In [19]:
training_df[training_df.wheeloccupancy==1]

,route,times,days,wheeloccupancy,departure,destination
4,1,11:00,mon,1,"""Westmoreland Street, stop 319""","""The Village, stop 119"""
6,1,13:00,mon,1,"""DCU St Patrick's, stop 21""","""Temple Street, stop 51"""
20,1,15:00,tues,1,"""Whitehall College, stop 203""","""Wellpark Avenue, stop 85"""
32,1,15:00,wed,1,"""Whitehall Church, stop 1620""","""Westmoreland Street, stop 319"""
79,11,14:00,tues,1,"""Merton Drive, stop 886""","""Dean Swift Road, stop 35"""
...,...,...,...,...,...,...
108904,84A,11:00,mon,1,"""St John's Church, stop 3080""","""Old Connaught Avenue, stop 4129"""
108915,84A,10:00,tues,1,"""Castle Street, stop 4130""","""Merrion Gates, stop 476"""
108939,84A,10:00,thurs,1,"""St James's Church, stop 4203""","""Shrewsbury Lawn, stop 3130"""
108964,84X,11:00,mon,1,"""St Anthony Church, stop 4268""","""Greystones Station, stop 4283"""


In [20]:
testing_df[testing_df.wheeloccupancy==1]

,route,times,days,wheeloccupancy,departure,destination
86,11,09:00,wed,1,"""Drummartin Park, stop 3024""","""Parnell Square West, stop 10"""
126,116,13:00,mon,1,"""Teresian School, stop 771""","""Stillorgan College, stop 2011"""
134,116,09:00,tues,1,"""Stillorgan Luas, stop 7057""","""Carmanhall Road, stop 3181"""
182,118,09:00,mon,1,"""St Johns Ambulance, stop 907""","""Donnybrook Garda Stn, stop 774"""
243,120,10:00,mon,1,"""Naomh Fionnbarra GAA, stop 4485""","""St Finbar's Road, stop 829"""
...,...,...,...,...,...,...
36268,84X,11:00,wed,1,"""Dawson Street, stop 792""","""Chapel Road, stop 4216"""
36308,9,15:00,mon,1,"""Beechfield Avenue, stop 2432""","""Mount Argus Church, stop 2391"""
36329,9,12:00,wed,1,"""Cromwellsfort Road, stop 2416""","""Clancy Avenue, stop 4376"""
36339,9,10:00,thurs,1,"""Lorcan O'Toole Park, stop 2436""","""Harrington Street, stop 1350"""


In [21]:
training_df.drop("departure", axis="columns", inplace=True)

In [22]:
training_df.drop("destination", axis="columns", inplace=True)

In [23]:
testing_df.drop("destination", axis="columns", inplace=True)

In [24]:
testing_df.drop("departure", axis="columns", inplace=True)

In [25]:
testing_df.dtypes

route             object
times             object
days              object
wheeloccupancy     int64
dtype: object

In [26]:
training_df.dtypes

route             object
times             object
days              object
wheeloccupancy     int64
dtype: object

In [27]:
training_df.head()

,route,times,days,wheeloccupancy
0,1,07:00,mon,0
1,1,08:00,mon,0
2,1,09:00,mon,0
3,1,10:00,mon,0
4,1,11:00,mon,1


In [28]:
testing_df.head()

,route,times,days,wheeloccupancy
0,1,07:00,mon,0
1,1,08:00,mon,0
2,1,09:00,mon,0
3,1,10:00,mon,0
4,1,11:00,mon,0


In [29]:

X_train_route_all = training_df.drop("wheeloccupancy", axis="columns")
y_train_route_all = training_df["wheeloccupancy"]

In [30]:
X_test_route_all = testing_df.drop("wheeloccupancy", axis="columns")
y_test_route_all = testing_df["wheeloccupancy"]

In [31]:
import numpy
unique, counts = numpy.unique(y_train_route_all, return_counts=True)
dict(zip(unique, counts))

{0: 103970, 1: 5111}

In [32]:
from sklearn.preprocessing import OneHotEncoder
ohe_all=OneHotEncoder(handle_unknown='ignore')
X_train_hot_route_all=ohe_all.fit_transform(X_train_route_all)
X_test_hot_route_all=ohe_all.transform(X_test_route_all)
#test=ohe.transform(test)

In [33]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="minority")
X_sm_all, y_sm_all = smote.fit_sample(X_train_hot_route_all,y_train_route_all)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [34]:
import numpy
unique, counts = numpy.unique(y_sm_all, return_counts=True)
dict(zip(unique, counts))

{0: 103970, 1: 103970}

In [35]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=3, random_state=0)
#clf.fit(trainingdata_encode_shuffle, train_label_shuffle)
#print(clf.predict([[0, 0, 0, 0]]))
#my second version
#clf.fit(trainingdata_encode, train_label)

#bus_df_encode variables
clf.fit(X_sm_all, y_sm_all)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [36]:
# this is the right step
pred_rou_all = clf.predict(X_test_hot_route_all)

In [37]:
pred_rou_all[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [38]:
y_test_route_all[:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: wheeloccupancy, dtype: int64

In [39]:
#this is the right step for the classification report
from sklearn.metrics import  confusion_matrix, classification_report

print(classification_report (y_test_route_all, pred_rou_all) )

              precision    recall  f1-score   support

           0       0.98      0.45      0.62     34688
           1       0.07      0.86      0.13      1673

    accuracy                           0.47     36361
   macro avg       0.53      0.65      0.37     36361
weighted avg       0.94      0.47      0.60     36361

